유전학 연구실에서 사용할 미국 서베이 데이터를 모아서 데이터프레임으로 만드는 작업이다.

데이터 소스는 [CDC-BRFSS Survey](https://www.cdc.gov/brfss/annual_data/annual_2016.html)이며 raw데이터로 아스키 파일을 제공한다.

여기서 약 10개 년도에 대해서만 조사를 수행하였으므로 따로 크롤러는 만들지 않는다.

## 1. 데이터 불러오기

먼저 데이터를 불러온다. 이 아스키 파일들은 하나당 100메가바이트 정도 되는 꽤 큰 파일이다.

각 행은 한 명에 대해 수행한 서베이 내용이며, 고정폭으로 변수에 대한 값들이 주어져있다.

In [53]:
import os
import numpy as np
import pandas as pd

os.chdir("/opt/notebooks/Playground/uidam/ASCII")
os.listdir()

['CNTY09.ASC',
 'CNTY02.ASC',
 'CNTY04.ASC',
 'CNTY10.ASC',
 'CNTY05.ASC',
 'CNTY12.ASC',
 'CNTY06.ASC',
 '.ipynb_checkpoints',
 'Process_1',
 'CNTY11.ASC',
 'index.csv',
 'CNTY03.ASC',
 'CNTY08.ASC']

인덱스 파일을 먼저 불러온다. 이 파일은 몇 번째 값이 어떤 변수인지를 알려주는 말그대로 인덱스 파일이다.

아래 예에서 미국 주(STATE)에 대한 정보는 각 파일의 1-2번째 문자열에 있다고 해석하면 된다.

In [54]:
index_file = pd.read_csv('../index-2.csv')
index_file

,Type,2002,2003,2004,2005,2006,2008,2009,2010,2011,2012
0,STATE,1-2,1-2,1-2,1-2,1-2,1-2,1-2,1-2,1-2,1-2
1,DROCCDY_,880-882,383-385,371-373,413-415,420-422,415-417,519-521,506-508,387-389,196-198
2,BINGE,884,387,375,416,423,418,522,509,390,199
3,_SMOKER2,878,381,369,411,372,367,471,458,201,193
4,SEX,149,140,119,128,110,118,123,118,107,84
5,_MRACE,913-914,413-414,409-410,442-443,388-389,383-384,487-488,474-475,217-218,163-164
6,RACE2,915,415,411,444,395,385,489,476,219,165
7,AGEG5YR,920-921,420-421,417-418,450-451,396-397,391-392,495-496,482-483,225-226,169-170
8,COUNTYNAME,958-989,491-540,478-527,542-591,473-522,468-507,618-667,559-608,413-415,232-281


불러올 아스키 파일은 아래와 같은 형태이다. 이를 `pandas`의 `read.fwf` 메서드를 통해 불러올 것이다.

In [11]:
with open('CNTY03.ASC') as r :
    one_line = r.readline()
one_line

'01010108200301082003101110200300002120030000210102010128888  111233 141302301302102102103213142 112  888    2 75218     18847041401354092 22 11112 1222 1058202     2                                                                            1111111117113121222222     22                                                    10101011342.17563107217512122001000001020020030007174113114100021000000001 11          1010111111122057145063643036325000001331211   07312122 923.937412.700184720611062Jefferson County                                  073                       1\n'

## 2. 데이터 변환 -  ASCII->CSV

이제 아스키를 다루기 쉬운 CSV 형태로 변환해준다.

In [55]:
sorted(os.listdir())[1:11]

['CNTY02.ASC',
 'CNTY03.ASC',
 'CNTY04.ASC',
 'CNTY05.ASC',
 'CNTY06.ASC',
 'CNTY08.ASC',
 'CNTY09.ASC',
 'CNTY10.ASC',
 'CNTY11.ASC',
 'CNTY12.ASC']

In [56]:
# 열 이름
col_names = ['STATE', 'DROC', 'BINGE', 'SMOKER2', 'SEX', 'MRACE', 'RACE2', 'AGE5YR', 'COUNTY']

# 아스키파일만 지정해준다
file_list = sorted(os.listdir())[1:11]

k = 1
year = 2002

for i in file_list:
    print("File ", i, " Start.")

    indices = []
    
    # 인덱스가 여러개인 경우와 하나인 경우로 나누어 실행한다
    for j in range(9):
        ind = index_file.iloc[:, k]
        if ('-' in ind[j]):
            start, end = map(int, ind[j].split('-'))
            indices.append([start - 1, end])

        else:
            start = int(ind[j]) - 1
            end = int(ind[j]) - 1
            indices.append([start, end + 1])

    # read_fwf 메서드로 임시 데이터프레임을 만든다.
    temp_file = pd.read_fwf(i, colspecs=indices, na_values="NA", dtype='str', names=col_names)

    # County 이름의 앞뒤 공백을 제거한다
    temp_file.iloc[:, 8] = list(map(str.strip, temp_file.iloc[:, 8]))
    # Year라는 새로운 열을 만들어준다
    temp_file['Year'] = year
    
    
    print("Shape: ", temp_file.shape)
    new_name = "Process_1/Pandas" + str(year) + ".csv"
    temp_file.to_csv(new_name, index=False)
    print("File ", i, " Finished.")
    k += 1
    year += 1

File  CNTY02.ASC  Start.
Shape:  (94624, 10)
File  CNTY02.ASC  Finished.
File  CNTY03.ASC  Start.
Shape:  (102698, 10)
File  CNTY03.ASC  Finished.
File  CNTY04.ASC  Start.
Shape:  (133364, 10)
File  CNTY04.ASC  Finished.
File  CNTY05.ASC  Start.
Shape:  (171683, 10)
File  CNTY05.ASC  Finished.
File  CNTY06.ASC  Start.
Shape:  (170995, 10)
File  CNTY06.ASC  Finished.
File  CNTY08.ASC  Start.
Shape:  (206029, 10)
File  CNTY08.ASC  Finished.
File  CNTY09.ASC  Start.
Shape:  (216379, 10)
File  CNTY09.ASC  Finished.
File  CNTY10.ASC  Start.
Shape:  (232530, 10)
File  CNTY10.ASC  Finished.
File  CNTY11.ASC  Start.
Shape:  (243403, 10)
File  CNTY11.ASC  Finished.
File  CNTY12.ASC  Start.
Shape:  (223988, 10)
File  CNTY12.ASC  Finished.


하나의 파일당 10만~20만개의 열을 갖는 데이터프레임을 생성해주었다. 데이터프레임을 살펴보면 아래와 같다.

In [33]:
temp_file.head()

,STATE,DROC,BINGE,SMOKER2,SEX,MRACE,RACE2,AGE5YR,COUNTY,Year
0,01,000,1,3,2,01,1,11,Jefferson County,2011
1,01,000,1,3,2,01,1,03,Jefferson County,2011
2,01,000,1,4,2,01,1,13,Jefferson County,2011
3,01,000,1,2,2,01,1,08,Jefferson County,2011
4,01,900,9,9,2,01,1,08,Jefferson County,2011


## 3. 데이터 변환 -  범주형 변수 카운트

우리가 알고 싶은 것은 각 그룹의 평균 음주량, 과음 빈도, 흡연 빈도이다.

하지만 위 데이터는 행 하나하나가 개인의 것이므로 이를 그룹화하여 빈도를 세줘야 한다.

In [60]:
os.chdir("Process_1/")
file_list = sorted(os.listdir())
print(file_list)

year_list = [2002, 2003, 2004, 2005, 2006, 2008, 2009, 2010, 2012]
tot_dat = pd.DataFrame()

for i, j in zip(file_list, year_list):
    print(i)
    count = 0

    temp_dat = pd.read_csv(i)

    # 년도, 주, 카운티, 인종, 연령, 성별로 그룹을 만든다.
    tot_group = temp_dat.groupby(['Year', 'STATE', 'COUNTY', 'RACE2', 'AGE5YR', 'SEX'])
    # 똑같이 그룹을 만들되 BINGE==2 (과음을 하는 사람)을 기준으로 그룹
    group_binge = temp_dat[temp_dat['BINGE'] == 2].groupby(['Year', 'STATE', 'COUNTY', 'RACE2', 'AGE5YR', 'SEX'])
    # 똑같이 그룹을 만들되 SMOKER2==1, 2, 3 (흡연을 하는 사람)을 기준으로 그룹
    group_smoke = temp_dat[temp_dat['SMOKER2'].isin([1, 2, 3])].groupby(['Year', 'STATE', 'COUNTY', 'RACE2', 'AGE5YR', 'SEX'])
    # 똑같이 그룹을 만들되 DROC != 900 (음주를 하는 사람) 기준으로 그룹
    group_droc = temp_dat[temp_dat['DROC'] != 900].groupby(['Year', 'STATE', 'COUNTY', 'RACE2', 'AGE5YR', 'SEX'])

    # 위에서 만든 그룹들을 데이터프레임으로 만든 뒤, 그룹을 기준으로 통합한다.
    df_group = pd.DataFrame({'count': tot_group['Year'].count()}).reset_index()
    df_binge = pd.DataFrame({'binge': group_binge['BINGE'].count()}).reset_index()
    df_smoke = pd.DataFrame({'smoke': group_smoke['SMOKER2'].count()}).reset_index()
    df_droc = pd.DataFrame({'droc': group_droc['DROC'].sum()}).reset_index()

    # 데이터프레임 통합
    new_df = pd.DataFrame()
    new_df = pd.merge(df_group, df_binge, how='outer')
    new_df = pd.merge(new_df, df_smoke, how='outer')
    new_df = pd.merge(new_df, df_droc, how='outer')
    new_df = new_df.fillna(0)

    print(new_df.shape)

    tot_dat = pd.concat([tot_dat, new_df])

tot_dat.shape

['Pandas2002.csv', 'Pandas2003.csv', 'Pandas2004.csv', 'Pandas2005.csv', 'Pandas2006.csv', 'Pandas2007.csv', 'Pandas2008.csv', 'Pandas2009.csv', 'Pandas2010.csv', 'Pandas2011.csv']
Pandas2002.csv
(12046, 10)
Pandas2003.csv
(13155, 10)
Pandas2004.csv


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(16949, 10)
Pandas2005.csv
(20435, 10)
Pandas2006.csv
(20315, 10)
Pandas2007.csv
(23007, 10)
Pandas2008.csv
(24232, 10)
Pandas2009.csv
(26596, 10)
Pandas2010.csv
(22750, 10)


(179485, 10)

In [61]:
tot_dat.head()

,Year,STATE,COUNTY,RACE2,AGE5YR,SEX,count,binge,smoke,droc
0,2002,1.0,Jefferson County,1,1.0,1.0,11,6.0,6.0,336.0
1,2002,1.0,Jefferson County,1,1.0,2.0,10,5.0,4.0,170.0
2,2002,1.0,Jefferson County,1,2.0,1.0,6,5.0,5.0,238.0
3,2002,1.0,Jefferson County,1,2.0,2.0,15,1.0,2.0,133.0
4,2002,1.0,Jefferson County,1,3.0,1.0,11,6.0,6.0,343.0


이렇게하여 위와 같은 최종 데이터프레임을 얻었다.

첫 행을 해석해보면 2002년에 1번(Alabama) 주, Jefferson 카운티의 인종이 1이고 연령집단이 1, 성별이 남성인 그룹은 11명(count)이고, 그 중 6명이 과음(binge)을 하며, 5명이 흡연자(smoke)이고, 평균 음주량이 336/100(droc)인 특성을 가지고 있다.

이제 이 자료들을 시각화하여 분석하고, 사용 가능한 데이터인 경우 모델에 투입한다.